In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import scipy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset, Dataset


import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

import utils

from image_list import ImageList

In [2]:
# Define dataset
attack_dataset = ImageList(
    filename="data/attack_dataset.txt", 
    root="data/IMAGENET/",
    color=True,
    # transform=transforms.ToTensor(),
    transform = transforms.Compose([transforms.Resize((32, 32)),
                                    ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                   ]),
    return_filename=True
)

# Create DataLoader
attack_dataloader = DataLoader(attack_dataset, batch_size=64, shuffle=False)

In [6]:
def extract_model(model, dataloader, output_file, save_softlabels=False, device="cuda" if torch.cuda.is_available() else "cpu", n_bacthes=None):
    """
    Faz inferência com um modelo PyTorch e salva o caminho da imagem e a classe inferida em um arquivo de saída.

    Args:
        model (torch.nn.Module): Modelo PyTorch treinado.
        dataloader (torch.utils.data.DataLoader): DataLoader com imagens e seus caminhos.
        output_file (str): Caminho do arquivo de saída.
        save_softlabels (bool, optional): Se True, salva as probabilidades para todas as classes. Default: False.
        device (str, optional): Dispositivo para inferência ("cuda" ou "cpu"). Default: "cuda" se disponível.
    """
    model.to(device)
    model.eval()  # Modo de inferência
    total = 0
    with open(output_file, "w") as f, torch.no_grad():
        for images, _labels, paths in dataloader:  # Assumindo que o dataset retorna (imagem, caminho)
            if(n_bacthes != None and total == n_bacthes):
                break
            total+=1
            images = images.to(device)
            outputs = model(images)
    
            if save_softlabels:
                softlabels = torch.nn.functional.softmax(outputs, dim=1).cpu().numpy().tolist()
                for path, probs in zip(paths, softlabels):
                    probs_str = " ".join(map(str, probs))  # Converte lista de probabilidades para string
                    f.write(f"{path} {probs_str}\n")
            else:
                _, preds = torch.max(outputs, 1)  # Obtém a classe com maior probabilidade
                for path, pred in zip(paths, preds.cpu().numpy()):
                    f.write(f"{path} {pred}\n")  # Salva caminho da imagem e classe inferida

    print(f"Inferência concluída. Resultados salvos em {output_file}")

In [3]:
#Melhorado pelo gpt
def extract_model(model, dataloader, output_file, device="cuda" if torch.cuda.is_available() else "cpu", n_batches=None):
    """
    Extracts soft labels from a trained model and saves them to a file.

    Args:
        model (torch.nn.Module): Trained PyTorch model.
        dataloader (torch.utils.data.DataLoader): DataLoader with (image, label, path).
        output_file (str): Path to the output file.
        device (str, optional): Inference device ("cuda" or "cpu"). Default: "cuda" if available.
        n_batches (int, optional): Number of batches to process. If None, processes all.
    """
    model.to(device)
    model.eval()  # Inference mode
    total = 0

    with open(output_file, "w") as f, torch.no_grad():
        for images, _labels, paths in dataloader:
            if n_batches is not None and total >= n_batches:
                break
            total += 1

            images = images.to(device)
            outputs = model(images)

            softlabels = torch.nn.functional.softmax(outputs, dim=1).cpu().numpy().tolist()
            for path, probs in zip(paths, softlabels):
                probs_str = " ".join(map(str, probs))
                f.write(f"{path} {probs_str}\n")

    print(f"Inference completed. Results saved in {output_file}")


In [20]:
model_mnist = utils.create_resnet_model('./pths/mnist_32.pth')
model_svhn = utils.create_resnet_model('./pths/svhn_32.pth')


In [21]:
# hardlabels = only 1 target
# extract_model(model_mnist, attack_dataloader, "extracted_mnist.txt")
# extract_model(model_svhn, attack_dataloader, "extracted_svhn.txt")

# softlabels
extract_model(model_mnist, attack_dataloader, "data/extracted_soft_mnist.txt", save_softlabels=True)
extract_model(model_svhn, attack_dataloader, "data/extracted_soft_svhn.txt", save_softlabels=True)

# extract_model(model_svhn, attack_dataloader, "teste.txt", save_softlabels=True, n_batchs=1)

Inferência concluída. Resultados salvos em data/extracted_soft_mnist.txt
Inferência concluída. Resultados salvos em data/extracted_soft_svhn.txt


In [5]:
model_combined = utils.create_resnet_model('./pths/combined_32.pth')
extract_model(model_combined, attack_dataloader, "data/extracted_soft_combined.txt", True)

Inference completed. Results saved in data/extracted_soft_combined.txt


In [7]:
model_combined = utils.create_resnet_model('./pths/combined_32.pth')
extract_model(model_combined, attack_dataloader, "data/extracted_combined.txt", False)

Inferência concluída. Resultados salvos em data/extracted_combined.txt
